In [1]:
%run 2-0-det2-training-common.ipynb

Thu Feb 15 16:56:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:D8:00.0 Off |                    0 |
| N/A   46C    P0              28W /  70W |      2MiB / 15360MiB |      7%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# RETINANET: SINGLE STAGE: set cfg backbone retinanet class number to 4, patient

In [2]:
# Import necessary modules and classes
#from .detectron2.tools.train_net import Trainer  # Uncomment if this is required
#from detectron2.engine import DefaultTrainer  # Uncomment if this is required
# select from modelzoo here: https://github.com/facebookresearch/detectron2/blob/master/MODEL_ZOO.md#coco-object-detection-baselines
from detectron2.config import get_cfg
#from detectron2.evaluation.coco_evaluation import COCOEvaluator  # Uncomment if this is required
import os
# Create a configuration object
cfg = get_cfg()

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.001 # confidence
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.6 # iou

cfg.MODEL.ROI_HEADS.IOU_THRESHOLDS = [0.2, 0.95]
print(cfg)

# cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_101_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/retinanet_R_101_FPN_3x.yaml")

cfg.OUTPUT_DIR = "output/retina_fpn"
cfg.MODEL.RETINANET.NUM_CLASSES=4
cfg.MODEL.ROI_HEADS.NUM_CLASSES=4
cfg.MODEL.ROI_BOX_HEAD.FED_LOSS_NUM_CLASSES=4
cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES=4

# Set training and validation datasets
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)

# Set number of data loading workers
cfg.DATALOADER.NUM_WORKERS = 4


# Set batch size and base learning rate
cfg.SOLVER.IMS_PER_BATCH = 8
cfg.SOLVER.BASE_LR = 0.05

# Set warm-up iterations, maximum iterations, steps, and gamma for the learning rate scheduler
cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.MAX_ITER = 20000  # Adjust up if val mAP is still rising, adjust down if overfit
cfg.SOLVER.STEPS = (1000, 1500)
# cfg.SOLVER.GAMMA = 0.05

# Set batch size per image and number of classes for ROI heads
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64

cfg.MODEL.RETINANET.NUM_CLASSES=4
# Set evaluation period during training
cfg.TEST.EVAL_PERIOD = 500

# Create output directory if it does not exist
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Create a CocoTrainer object
trainer = CocoTrainer(cfg)

if cfg.TEST.AUG.ENABLED:
  trainer.register_hooks(
    [hooks.EvalHook(0, lambda: trainer.test_with_TTA(cfg, trainer.model))]   #this block uses a hook to run evalutaion periodically
  )                                           #https://detectron2.readthedocs.io/en/latest/modules/engine.html#detectron2.engine.hooks.EvalHook


# Resume training or load from a checkpoint
trainer.resume_or_load(resume=True)
# trainer.resume_or_load(resume=False)
# Train the model

Loading config /work/van-speech-nlp/jindaznb/j-vis/detectron2/detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


CUDNN_BENCHMARK: False
DATALOADER:
  ASPECT_RATIO_GROUPING: True
  FILTER_EMPTY_ANNOTATIONS: True
  NUM_WORKERS: 4
  REPEAT_THRESHOLD: 0.0
  SAMPLER_TRAIN: TrainingSampler
DATASETS:
  PRECOMPUTED_PROPOSAL_TOPK_TEST: 1000
  PRECOMPUTED_PROPOSAL_TOPK_TRAIN: 2000
  PROPOSAL_FILES_TEST: ()
  PROPOSAL_FILES_TRAIN: ()
  TEST: ()
  TRAIN: ()
GLOBAL:
  HACK: 1.0
INPUT:
  CROP:
    ENABLED: False
    SIZE: [0.9, 0.9]
    TYPE: relative_range
  FORMAT: BGR
  MASK_FORMAT: polygon
  MAX_SIZE_TEST: 1333
  MAX_SIZE_TRAIN: 1333
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN: (800,)
  MIN_SIZE_TRAIN_SAMPLING: choice
  RANDOM_FLIP: horizontal
MODEL:
  ANCHOR_GENERATOR:
    ANGLES: [[-90, 0, 90]]
    ASPECT_RATIOS: [[0.5, 1.0, 2.0]]
    NAME: DefaultAnchorGenerator
    OFFSET: 0.0
    SIZES: [[32, 64, 128, 256, 512]]
  BACKBONE:
    FREEZE_AT: 2
    NAME: build_resnet_backbone
  DEVICE: cuda
  FPN:
    FUSE_TYPE: sum
    IN_FEATURES: []
    NORM: 
    OUT_CHANNELS: 256
  KEYPOINT_ON: False
  LOAD_PROPOSALS: Fals

/work/van-speech-nlp/det2env/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 3, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[02/15 16:57:13 d2.engine.hooks]: Loading scheduler from state_dict ...


In [3]:
# trainer.train()

In [4]:
cfg.OUTPUT_DIR

'output/retina_fpn'

In [5]:
#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset



cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_test", cfg, False, output_dir="./output/")
# evaluator.params.iouThrs = 
val_loader = build_detection_test_loader(cfg, "my_dataset_test")
inference_on_dataset(trainer.model, val_loader, evaluator)

[02/15 16:57:13 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from output/retina_fpn/model_final.pth ...
WARNING [02/15 16:57:14 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[02/15 16:57:14 d2.data.datasets.coco]: Loaded 67 images in COCO format from /work/van-speech-nlp/jindaznb/j-vis/Forestfire2024-1/test/_annotations.coco.json
[02/15 16:57:14 d2.data.build]: Distribution of instances among all 4 categories:
|  category  | #instances   |   category    | #instances   |  category  | #instances   |
|:----------:|:-------------|:-------------:|:-------------|:----------:|:-------------|
| Alive Tree | 6961         | Beetle-Fire.. | 752          | Dead Tree  | 531          |
|   Debris   | 512          |               |              |            |              |
|   total    | 8756         |               |              |            |              |
[02/15 16:57:14 d

/work/van-speech-nlp/det2env/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[02/15 16:57:17 d2.evaluation.evaluator]: Inference done 11/67. Dataloading: 0.0008 s/iter. Inference: 0.0924 s/iter. Eval: 0.0003 s/iter. Total: 0.0935 s/iter. ETA=0:00:05
[02/15 16:57:22 d2.evaluation.evaluator]: Inference done 64/67. Dataloading: 0.0012 s/iter. Inference: 0.0942 s/iter. Eval: 0.0003 s/iter. Total: 0.0958 s/iter. ETA=0:00:00
[02/15 16:57:22 d2.evaluation.evaluator]: Total inference time: 0:00:06.011929 (0.096967 s / iter per device, on 1 devices)
[02/15 16:57:22 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:05 (0.094182 s / iter per device, on 1 devices)
[02/15 16:57:22 d2.evaluation.coco_evaluation]: COCO format ...
[02/15 16:57:22 d2.evaluation.coco_evaluation]: Saving results to ./output/coco_instances_results.json
[02/15 16:57:22 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.24s)
creating index...
index created!
[02/15 16:57:23 d2.evaluation.fast_eval_api]: Evaluat

OrderedDict([('bbox',
              {'AP': 28.272632926607717,
               'AP50': 36.22430194443408,
               'AP75': 23.348718939763113,
               'APs': 1.1881023318376758,
               'APm': 36.73879246484007,
               'APl': 69.26232059237715,
               'AP-Alive Tree': 34.46712264607976,
               'AP-Beetle-Fire Tree': 22.8296923692859,
               'AP-Dead Tree': 27.44900599581434,
               'AP-Debris': 28.34471069525088})])

In [6]:
%ls ./output/

coco_instances_results.json  instances_predictions.pth
faster-rcnn/                 retina_fpn/


In [7]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = ("my_dataset_test", )
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
predictor = DefaultPredictor(cfg)
test_metadata = MetadataCatalog.get("my_dataset_test")

[02/15 16:57:24 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from output/retina_fpn/model_final.pth ...


In [8]:
from detectron2.utils.visualizer import ColorMode
import glob

for imageName in glob.glob('/content/test/*jpg'):
  im = cv2.imread(imageName)
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1],
                metadata=test_metadata, 
                scale=0.8
                 )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])